In [1]:
import os
from tqdm import tqdm
import pandas         as pd
import zipfile
import requests

from bs4              import BeautifulSoup
from time             import sleep
from datetime         import datetime
from xml.etree        import ElementTree

# Dados Estações

In [61]:
url = 'https://gbfs.lyft.com/gbfs/2.3/bay/en/station_information.json'

response = requests.get( url )

if response.status_code == 200:
    data = response.json()
    stations = data['data']['stations']
    
    df_station = pd.DataFrame( columns=['station_id', 'capacity', 'name', 'region_id', 'short_name', 'lat', 'lon'] )
    
    for station in stations:
        station.pop('rental_uris')
        df = pd.DataFrame( [station], dtype='object' )
        df_station = pd.concat( [df_station, df], ignore_index=True )
else:
    print( 'Não foi possível encontrar as informações, verifique o que pode estar ocorrendo' )

In [85]:
url = 'https://gbfs.lyft.com/gbfs/2.3/bay/en/system_regions.json'
response = requests.get( url )

if response.status_code == 200:
    data = response.json()
    
    regions = data['data']['regions']
    
    df_regions = pd.DataFrame( columns=['region_id', 'name'] )
    
    for region in regions:
        df = pd.DataFrame( [region], dtype='object' )
        df_regions = pd.concat( [df_regions, df], ignore_index=True )
else:
    print( 'Não foi possível encontrar as informações, verifique o que pode estar ocorrendo' )

In [88]:
df = df_station.merge( df_regions, how='left', on='region_id' )

In [93]:
df.columns = ['station_id', 'capacity', 'station_name', 'region_id', 'short_name', 'lat', 'lon', 'address', 'region']

In [95]:
date_time = datetime.fromtimestamp( data['last_updated'] ).strftime( '%Y%m%d' )

In [99]:
df_station

,station_id,capacity,name,region_id,short_name,lat,lon,address
0,c53990d7-f965-40f4-b305-3435e1c95a71,19,23rd St at Santa Clara St,5,SJ-M14,37.34648,-121.86857,NaN
1,ab8cc22e-0f34-4476-bf81-293cbbb2e69c,27,Kerley Dr at Rosemary St,5,SJ-F10,37.360854,-121.906834,NaN
2,30429aed-9a47-4fd9-87fa-0db835aa8265,19,17th St at Santa Clara St,5,SJ-L13,37.343985,-121.874385,NaN
3,ed707a89-a68d-4921-a4cb-16c268e45a5b,23,San Fernando St at 7th St,5,SJ-M11-2,37.337122,-121.883215,NaN
4,6994e1e6-bb3c-4309-8207-52e6004a7302,23,Newbury Park Dr at King Rd,5,SJ-H17,37.365536,-121.867966,NaN
...,...,...,...,...,...,...,...,...
544,1880193329986742718,4,Taraval St: 41st St to 40th St,NaN,NaN,37.742093,-122.498529,"3008 Taraval St, San Francisco, CA 94116, Unit..."
545,1880193329986742728,2,Quintara at 21st,NaN,NaN,37.748508,-122.478419,"2101 21st Ave, San Francisco, CA 94116, United..."
546,1841972814951361202,8,Stowe Lake,NaN,NaN,37.770938,-122.477116,"50 Stow Lake Dr, San Francisco, CA 94118, Unit..."
547,1841972814951361200,5,Bowling Green,NaN,NaN,37.76873,-122.459429,"Bowling Green Dr, San Francisco, CA 94118, Uni..."


In [97]:
df.to_csv( f'data/{date_time}_station-info.csv', index=False )

# Dados Metereológicos

In [ ]:
coordenadas_locais = {
    'San Francisco': '37.7749,-122.4194',
    'San Jose': '37.3394,-121.895',
    'East Bay': '37.8044,-122.2708'
}

url = 'https://open-meteo.com/en/docs/historical-weather-api#latitude=37.7749&longitude=-122.4194&start_date=2023-01-01&end_date=2024-01-07&hourly=temperature_2m,relative_humidity_2m,precipitation,snowfall,wind_speed_10m,soil_temperature_7_to_28cm,is_day,sunshine_duration&daily=sunrise,sunset,shortwave_radiation_sum&timezone=America%2FLos_Angeles'


In [100]:
lat = '37.761047'
lon = '-122.432642'
start_date = '2022-01-01'
end_date = '2022-02-01'

url = f'https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date={start_date}&end_date={end_date}&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,is_day,sunshine_duration,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant&daily=weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,daylight_duration,sunshine_duration,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration&timezone=America%2FLos_Angeles&models=best_match,ecmwf_ifs,era5_seamless,era5,era5_land,cerra'

response = requests.get( url )

In [103]:
data = response.json()

In [126]:
df_weather_hourly = pd.DataFrame( data['hourly'] )

df_weather_hourly = df_weather_hourly.dropna( axis=1, how='all' )

In [138]:
df_weather_daily = pd.DataFrame( data['daily'] )

df_weather_daily = df_weather_daily.dropna( axis=1, how='all' )
df_weather_daily

,time,weather_code_archive_best_match,temperature_2m_max_archive_best_match,temperature_2m_min_archive_best_match,temperature_2m_mean_archive_best_match,apparent_temperature_max_archive_best_match,apparent_temperature_min_archive_best_match,apparent_temperature_mean_archive_best_match,sunrise_archive_best_match,sunset_archive_best_match,...,wind_direction_10m_dominant_era5,shortwave_radiation_sum_era5,et0_fao_evapotranspiration_era5,temperature_2m_max_era5_land,temperature_2m_min_era5_land,temperature_2m_mean_era5_land,sunrise_era5_land,sunset_era5_land,daylight_duration_era5_land,precipitation_hours_era5_land
0,2022-01-01,2,9.1,3.5,6.3,6.2,0.5,3.4,2022-01-01T08:25,2022-01-01T18:01,...,135,10.33,1.23,10.5,3.3,6.8,2022-01-01T08:25,2022-01-01T18:01,34575.59,0.0
1,2022-01-02,3,9.7,3.1,6.4,6.9,-0.6,3.4,2022-01-02T08:25,2022-01-02T18:02,...,351,9.27,1.11,11.3,0.4,5.6,2022-01-02T08:25,2022-01-02T18:02,34615.57,0.0
2,2022-01-03,53,12.0,7.0,9.9,8.5,4.3,6.4,2022-01-03T08:25,2022-01-03T18:03,...,269,3.60,0.79,12.2,6.0,9.2,2022-01-03T08:25,2022-01-03T18:03,34659.04,0.0
3,2022-01-04,53,12.5,9.4,11.4,11.1,7.9,9.7,2022-01-04T08:25,2022-01-04T18:03,...,205,3.83,0.58,13.5,8.6,11.4,2022-01-04T08:25,2022-01-04T18:03,34706.40,0.0
4,2022-01-05,3,13.1,9.9,11.1,12.6,8.0,10.3,2022-01-05T08:25,2022-01-05T18:04,...,195,2.71,0.47,13.1,9.3,11.2,2022-01-05T08:25,2022-01-05T18:04,34757.55,0.0
5,2022-01-06,3,11.4,9.0,10.2,10.9,7.8,9.2,2022-01-06T08:25,2022-01-06T18:05,...,147,3.66,0.54,12.0,9.7,10.8,2022-01-06T08:25,2022-01-06T18:05,34812.34,0.0
6,2022-01-07,51,12.1,7.9,10.3,9.4,5.8,7.9,2022-01-07T08:25,2022-01-07T18:06,...,200,5.75,0.84,12.8,8.0,10.3,2022-01-07T08:25,2022-01-07T18:06,34870.66,0.0
7,2022-01-08,3,11.8,6.5,8.9,9.7,4.2,6.9,2022-01-08T08:25,2022-01-08T18:07,...,97,8.77,1.05,13.6,4.9,8.8,2022-01-08T08:25,2022-01-08T18:07,34932.38,0.0
8,2022-01-09,3,12.7,6.6,8.7,10.6,2.2,5.9,2022-01-09T08:25,2022-01-09T18:08,...,15,10.46,1.29,14.3,3.8,8.0,2022-01-09T08:25,2022-01-09T18:08,34997.37,0.0
9,2022-01-10,3,14.8,7.2,10.0,12.6,4.5,7.6,2022-01-10T08:24,2022-01-10T18:09,...,34,10.64,1.44,15.6,6.3,9.9,2022-01-10T08:24,2022-01-10T18:09,35065.51,0.0


# Dados da Viagens

In [4]:
start_date = datetime.strptime('20220101', "%Y%m%d").date()
end_date = datetime.now().date()

url = 'https://s3.amazonaws.com/baywheels-data'

response = requests.get( url )

soup = BeautifulSoup( response.content, 'xml' )

files = soup.find_all('Key')

for file in tqdm(files):
    zip_file = file.text
    ref = zip_file.split('-')[0]

    zip_filepath = f'../data/{zip_file}'
    output_csv = f'../data/trip_data.csv'
    
    try:
        ref = datetime.strptime( ref, "%Y%m" ).date()

        if start_date <= ref <= end_date:
            
            download_file = f'{url}/{zip_file}'
            csv_file = zip_file[:-4]

            csv_filepath = f'../data/{csv_file}'

            if not os.path.exists( output_csv ):
                with open( output_csv, 'w' ) as f:
                    f.write( 'ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual\n' )
            
            os.system( f"wget -q -O {zip_filepath} {download_file}" )
            
            with zipfile.ZipFile( zip_filepath, 'r' ) as zip:
                zip.extract( csv_file, '../data' )

            for chunk in pd.read_csv( csv_filepath, chunksize= 50000 ):
                chunk.to_csv( output_csv, mode='a', header=False, index=False)
            
            os.remove( zip_filepath )
            os.remove( csv_filepath )

    except ValueError:
        pass

100%|███████████████████████████████████████████| 80/80 [05:39<00:00,  4.24s/it]


In [2]:
output_csv = f'../data/trip_data.csv'

df = pd.read_csv( output_csv )

df['member_casual'] = df['member_casual'].astype('category')
df['start_station_name'] = df['start_station_name'].astype('category')
df['end_station_name'] = df['end_station_name'].astype('category')
df['start_station_id'] = df['start_station_id'].astype('category')
df['end_station_id'] = df['end_station_id'].astype('category')
df['rideable_type'] = df['rideable_type'].astype('category')
df['started_at'] = pd.to_datetime( df['started_at'], format='mixed' )
df['ended_at'] = pd.to_datetime( df['ended_at'], format='mixed' )

df.to_parquet('../data/tripdata.parquet', engine='pyarrow')

del df

## Verificando as Estações

In [3]:
df = pd.read( output_csv, nrows=100000 )
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,0DD008BC62836D27,classic_bike,2022-01-13 19:12:23,2022-01-13 19:22:13,Washington St at Van Ness Ave,SF-E22,Natoma St at New Montgomery St,SF-G28-2,37.792980,-122.423302,37.786456,-122.399749,member
1,BE9F5C77F1BE0FBF,classic_bike,2022-01-15 15:58:55,2022-01-15 16:10:03,Washington St at Van Ness Ave,SF-E22,Natoma St at New Montgomery St,SF-G28-2,37.792980,-122.423302,37.786456,-122.399749,member
2,F0826402062D5A44,classic_bike,2022-01-07 20:34:47,2022-01-07 20:55:25,17th St at Dolores St,SF-N21,Scott St at Golden Gate Ave,SF-I19,37.763015,-122.426497,37.778999,-122.436861,member
3,EE9A8BB189061CFD,classic_bike,2022-01-31 16:38:25,2022-01-31 17:13:05,El Embarcadero at Grand Ave,OK-I9,Ninth St at Heinz Ave,BK-H3,37.808715,-122.249251,37.853907,-122.289698,member
4,772A8B545A8525C0,classic_bike,2022-01-28 17:28:22,2022-01-28 17:39:55,17th St at Dolores St,SF-N21,Octavia Blvd at Page St,SF-J22-1,37.763015,-122.426497,37.774018,-122.423809,casual


In [4]:
df.isna().sum()

ride_id                   0
rideable_type             0
started_at                0
ended_at                  0
start_station_name    12557
start_station_id      12557
end_station_name      11103
end_station_id        11103
start_lat                 0
start_lng                 0
end_lat                  56
end_lng                  56
member_casual             0
dtype: int64

In [8]:
df[df['start_station_name'].isna()]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
1091,95B9A7847720F560,electric_bike,2022-01-18 20:14:43,2022-01-18 20:19:39,NaN,NaN,18th St at Noe St,SF-O19,37.76,-122.42,37.761047,-122.432642,member
1092,2A653EF4CDCE1342,electric_bike,2022-01-10 08:54:10,2022-01-10 09:13:52,NaN,NaN,Terry Francois Blvd at Mission Bay Blvd N,SF-L31-1,37.81,-122.42,37.771767,-122.386689,member
1093,0665F2FA09B5C666,electric_bike,2022-01-08 20:33:19,2022-01-08 20:36:18,NaN,NaN,Terry Francois Blvd at Mission Bay Blvd N,SF-L31-1,37.77,-122.39,37.771767,-122.386689,casual
1547,6CAE891EE8D630A5,electric_bike,2022-01-30 10:59:49,2022-01-30 11:28:04,NaN,NaN,48th Ave at Cabrillo St,SF-J1,37.80,-122.43,37.772954,-122.509071,casual
1548,39568245D8151714,electric_bike,2022-01-20 17:49:45,2022-01-20 17:55:13,NaN,NaN,Laguna St at Hayes St,SF-J21,37.78,-122.42,37.776247,-122.426203,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,D1DE1F02C8764742,electric_bike,2022-01-03 11:11:53,2022-01-03 11:15:42,NaN,NaN,NaN,NaN,37.75,-122.43,37.760000,-122.430000,member
99996,E8C064DB8DAAB4CA,electric_bike,2022-01-03 12:24:21,2022-01-03 12:31:43,NaN,NaN,NaN,NaN,37.76,-122.43,37.750000,-122.440000,member
99997,B57B57704EF7FD33,electric_bike,2022-01-11 18:19:10,2022-01-11 18:27:01,NaN,NaN,NaN,NaN,37.76,-122.43,37.750000,-122.440000,member
99998,21DECB42D48AAD89,electric_bike,2022-01-11 17:08:20,2022-01-11 17:13:26,NaN,NaN,NaN,NaN,37.75,-122.44,37.760000,-122.430000,member
